In [1]:
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate
import requests
from urllib.parse import urlparse, urlunparse
from datasets import load_dataset
from utils import *
import getpass
import os
import re
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1 Recall from Common Crawl

In [2]:
urls =[
      'https://www.answers.com/other-engineering/What_is_coexisting_technology',
      'https://www.answers.com/engineering/Who_invented_the_brassiere']# Given the educational websites.

educational_websites=[]#load the  educational websites 100K
for url in urls:
    educational_websites.append({'url':url,'text':fetch_website_content(url),
                                'class':1})

请求出错: 504 Server Error: Fastly Internal for url: https://www.answers.com/engineering/Who_invented_the_brassiere


In [ ]:
#load the Common Crawl 100K
Common_Crawl_data= load_datasetset('jed351/Traditional-Chinese-Common-Crawl-Filtered ')
sublabel =[0]*len(Common_Crawl_data['train'])# add class for classifier
Common_Crawl_data =Common_Crawl_data.add_column(name='class',column=sublabel)

dataset = merge_data(Common_Crawl_data,educational_websites)
#train model
model = fastText()
#model.fit(dataset)

In [ ]:
urls_count=recall_tokens(Common_Crawl_data['train'] ,model , '100B') 

In [ ]:
#extarced
os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4")                  #should use GPT-3.5

useful_domains = recall_domains(llm,urls_count)#50K

In [ ]:
#new_data
new_data = []
for i in range(len(Common_Crawl_data['train'])):
    if Common_Crawl_data[i]['url'] in useful_domains:
        item = {'class':1,'url':Common_Crawl_data['train'][i]['url'],'text':Common_Crawl_data[i]['text'] }
    else:
        item = {'class':0,'url':Common_Crawl_data['train'][i]['url'],'text':Common_Crawl_data[i]['text'] }
    new_data.append(item)
#re-train
#model.fit(new_data)

In [ ]:
urls_count=recall_tokens(base_data['train'],model,'40B')
os.environ["GPT-4_API_KEY"] = getpass.getpass()

llm = ChatMistralAI(model="GPT-4")
useful_domains = recall_domains(llm,urls_count)

## ---------------------------自动执行-------------------------------------

In [ ]:
prefer_url=['biology.stackexchange.com',
'chemistry.stackexchange.com',
'physics.stackexchange.com',
'cs.stackexchange.com',
]
num_links = 2                # 输入： 每个网站需要提取的Q-A个数
with open('url_file.txt','a') as f:
    for url in prefer_url:
        base_url ='https://' + url
        links = scrape_links(base_url, num_links)
        for link in links:
            if link.startswith('https:'):
                f.write(link+'\n')
            else:
                f.write(base_url+link+'\n')

# 2 Q-A Pair Extraction

In [ ]:
url_list =[]
with open('url_file.txt','r') as f:
    for line in f.readlines():
        url_list.append(line.strip())

In [32]:
filter_results = clean_and_filter(url_list)

In [ ]:
#extarced
os.environ["OPENAI_API_KEY"] = getpass.getpass()

llm = ChatOpenAI(model="gpt-4")                  #should use Qwen-72B

In [ ]:
example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")
prefix = """
Given a string that contains a Q-A pair, extract and return the Q-A pair.
Don't return the Q-A pair in the example.
If the Q-A pairs does not have relations with math, science, engineering,return void.
"""  
examples=[{'question':r'''
How do you solve the equation 2x plus 30 equals equals 12x-50? - Answers    Math and Arithmetic\n    Algebra\n      Calculus\n            \nSubjects\n\n            Animals & Plants\n        \n\n            Arts & Entertainment\n        \n\n            Auto\n        \n\n            Beauty & Health\n        \n\n            Books and Literature\n        \n\n            Business\n        \n\n            Electronics\n        \n\n            Engineering & Technology\n        \n\n            Food & Drink\n        \n\n            History\n        \n\n            Hobbies\n        \n\n            Jobs & Education\n        \n\n            Law & Government\n        \n\n            Math\n        \n\n            People & Society\n        \n\n            Science\n        \n\n            Social Studies\n        \n\n            Sports\n        \n\n            Travel & Places\n        \n\n\n\n\nCreate0Log inSubjects>Math>CalculusHow do you solve the equation 2x plus 30 equals equals 12x-50?Updated: 10/16/2023Wiki User ∙ 13y agoStudy nowSee answers (2)Best AnswerCopySubtract 2x from each side: 30 = 10x - 50;Add 50 to each side: 80 = 10xDivide each side by 10: 8 = xWiki User ∙ 13y agoThis answer is:👍 Helpful (0)👎 Not Helpful (0) Add a CommentMore answersMandy Peters ∙ Lvl 2 ∙ 9mo agoCopy2x+x-2=7This answer is:👍 Helpful (1)👎 Not Helpful (0) Add a CommentAdd your answer:Earn +20 ptsQ: How do you solve the equation 2x plus 30 equals equals 12x-50?Write your answer...SubmitStill have questions?\n\n\n\n\n\n\n\n\n\n\n\n\nFind more answers\nAsk your question\n\n\n\n\n\nContinue Learning about CalculusSolve py plus qz equals pq by charpit method in partial differential equation?z=pqSolve the equation z2 plus 6z plus 24 equals 0?2z+6z+24=0\n8z+24=0\n8z= -24\nz= - 3What is 28 plus z equals 56?Set up the equation and solve for z:\n28 + z = 56 (next, subtract 28 from each side of the equaition\nto solve)\nz = 28Solve the equation for x plus 37 equals 98?why will the equations x+14=37 and x-14=37 have different\nsolutions for xHow do you solve this graphing equation y plus x equals 6?The solution consists of the infinite number of points on the\nline which is defined by y + x = 6.Related questionsSolve the equation k plus 3.4 equals 9?k equals 5.6Solve the equation x plus 13 equals -8?5W plus x equals 2y plus z for x?you can not solve this equationHow do you solve x plus 5 equals 9 plus x?The equation does not have a solution.Solve the equation x plus 32 equals 59?27Solve the equation 6m plus 15 equals 12?-0.5\n\n\n\n\n\nTrending Questions \nWhat month comes 4 months after April?\nWho is much higher Philippine peso or Venezuelan bolivar?\nIf i been taking birth control pills for 2 years and missed the first week of my birth control pills and had unprotected sex twice in that same week could i be pregnant?\nX plus y squared?\nPredict whether water would boil on top of a mountain at a higher or lower temperature than at sea level?\nWhat is the equation to 2 units down of the graph of y equals x?\nIs inpatient an adverb?\nHow do you describe how to translate the graph y equals radical x to obtain the graph of each function?\nHow many calorise are in 1 pound?\nHow many sixteenths in 1 fourth?\nHow long does it take to burn off one orange?\nWhat is the difference between continuous and contiguous memory allocation?\nWhen solving an inequality when is it necessary to change the direction of the inequality symbol?\nHow do you solve the equation 3parentheses x minus 8 parentheses plus 7 equal 2 x?\nWhat does to be faced with something mean?\nWhat does el calabo blanco mean in spanish?\nIs thick an adverb?\nWhat dose intolerable mean?\nWhat is the derivative of 4x?\nHow do you solve x squared minus 16?\n\n\nPreviously Viewed \nHow do you solve the equation 2x plus 30 equals equals 12x-50?\n\n\nUnanswered Questions \nIf the sum of interior of a polygon measures 13500?\nHow Can I Speed Up the Drying Time After Cleaning?\nWhy is a 24-hour coworking space in Bangalore ideal for freelancers and entrepreneurs?\nHow would you describe the alignment of my current life path with my objectives?\nWhat types of marble do you specialize in manufacturing?\nHow does the class encourage interaction and speaking practice among students?\nI am looking for a good diagnostic center in Guwahati, can anyone help me?\nWhat are the health benefits of incorporating mango milk into your daily diet?\n\n\n\n\n\n\n\n\n\nResources\n\n                    Leaderboard\n                \n\n                    All Tags\n                \n\n                    Unanswered\n                \n\n\nTop Categories\nAlgebra\nChemistry\nBiology\nWorld History\nEnglish Language Arts\nPsychology\nComputer Science\nEconomics\n\n\nProduct\nCommunity Guidelines\nHonor Code\nFlashcard Maker\nStudy Guides\nMath Solver\nFAQ\n\n\nCompany\nAbout Us\nContact Us\nTerms of Use\nPrivacy Policy\nDisclaimer\nCookie Policy\nIP Issues\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCopyright ©2024 Infospace Holdings LLC, A System1 Company. All Rights Reserved. The material on this site can not be reproduced, distributed, transmitted, cached or otherwise used, except with prior written permission of Answers.\n   

''',
'answer':r'''
Question: How do you solve the equation 2x + 30 = 12x - 50?
Answer:
Subtract 2𝑥 from both sides: 30=10𝑥−50
Add 50 to both sides: 80=10𝑥
Divide both sides by 10: 𝑥=8
'''  
},

]#more example better

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Now,do this for the document below,Question: {document}",
    prefix=prefix,
    input_variables=["document"],
)


In [ ]:
parser = StrOutputParser()

llm_pipe = llm | parser

In [ ]:
q_a_pairs=[]
for doc in filter_results:
    q_a = llm_pipe.invoke(input = prompt.format(document=doc))
    if 'Question:' in q_a:
        q_a_pairs.append(q_a)
        

In [ ]:
#remove

#we filter out all web pages that contain n-grams (n = 10) string matches with either the questions or answers.


# 3.Q-A Pair Refinement

In [ ]:
# Mixtral-22B×8  and Qwen-72B  in the artcle
os.environ["OPENAI_API_KEY"] = getpass.getpass()

llm = ChatOpenAI(model="gpt-4")    #should use Qwen-72B and Mixtral-22B×8 

In [63]:
example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")
prefix = """Give you a Q-A pair,you must refine the question and answer,make the question more clear and easy to understand,
keep the answer contain enough explanations ,with your knowledge in math, science, engineering.
You need to expand the question and answer,make the expression of both the question and the answer more precise,
explicitly provides the formula for calculating step by step,attempt to complete the intermediate reasoning steps leading to the given answer.
As a result, the question is clear and the answer is filled with relative knowledge.
""" 

examples=[{
  'question':  '''
  
Question:I am stucked on the following challenge: "If the line determined by two distinct points
(x1, y1) and (x2, y2) is not vertical, and therefore has slope (y2 − y1)/(x2 − x1), show that the pointslope form of its equation is the same regardless of which point is used as the given point." Okay, we
can separate (x0, y0) from the form to get:
y(x2 − x1) − x(y2 − y1) = y0(x2 − x1) − x0(y2 − y1)
But how exclude this point (x0, y0) and leave only x, y, x1, y1, x2, y2 in the equation? UPDATE: There
is a solution for this challenge:
(y1 − y2)x + (x2 − x1)y = x2y1 − x1y2
From the answer I found that
y2(x − x1) − y1(x − x2) = y(x2 − x1)
... but why this is true?

Answer:Thanks to saulspatz, the solution is to simply show that, whether we are using (x1, y1) or
(x2, y2) as the given point, the equation does not change.
So both equations:
y − y1 = m(x − x1)
y − y2 = m(x − x2)
reduce to:
(y1 − y2)x + (x2 − x1)y = x2y1 − x1y2.
  ''',
'answer':'''
Question:Demonstrate that the point-slope form of the equation of a line determined by two distinct
points (x1, y1) and (x2, y2), where the line is not vertical, remains consistent regardless of which point
is chosen as the reference. Why does the form of the equation not change when different reference
points are used?
Answer:
To address this challenge, we need to confirm that the point-slope equation of a line remains
consistent regardless of the point chosen to derive it. Given two points (x1, y1) and (x2, y2) with a
non-vertical line passing through them, the slope m of the line is y2−y1
x2−x1
.
The point-slope form of a line’s equation based on point (x1, y1) is:
y − y1 = m(x − x1)
Similarly, using point (x2, y2) gives:
y − y2 = m(x − x2)
These can be rewritten and compared to show their equivalence. Expanding both equations: - From the
first equation: y − y1 =
y2−y1
x2−x1
(x − x1) - From the second equation: y − y2 =
y2−y1
x2−x1
(x − x2)
We simplify these to find a common form. Multiplying both sides by (x2 − x1) and rearranging terms,
we find:
y2(x − x1) − y1(x − x2) = y(x2 − x1)
This confirms that both forms collapse to the same linear equation, showing the consistency of the
point-slope form irrespective of the chosen reference point. Thus, it is demonstrated that the structural
form of the equation remains unchanged, ensuring that the mathematical properties of the line—defined
by its slope and points through which it passes—are invariant to the choice of the reference point used
in its expression. This uniformity stems from the linearity and the constant slope defined by any two
points on a straight line.

'''  
},
{
   'question':"""
Question: Find the distance from the point 3
2
,
3
2
to the line −
37x
10 −
21y
10 −
7
2 = 0.
Answer: 61 拉 2
905 .
   """,
'answer':"""
Question: Calculate the distance from the point ( 3
2
,
3
2
) to the line defined by the equation
−
37
10 x −
21
10 y −
7
2 = 0.
Answer: To find the distance ’d’ from a point (x0, y0) to a line given by Ax + By + C = 0, we use
the formula:
d =
|Ax0 + By0 + C|
√
A2 + B2
.
For the point 
3
2
,
3
2

and the line −
37
10 x −
21
10 y −
7
2 = 0, substitute the coordinates into the formula:
d =
| − 37
10 ·
3
2 −
21
10 ·
3
2 −
7
2
|
拉
−
37
10 2 +

−
21
10 2
.
Simplifying further, the exact simplified distance ’d’ is 61拉 2
905 . Thus, the correct distance from the
point to the line is 61拉 2
905 . 
    """
},
{'question':
"""
Question: What are the two main ways that minerals are classified?
Answer: They can be classified by luster, streak, breakage, and hardness
"""
,
 'answer':"""
 Question: What are the primary criteria used for classifying minerals in mineralogy?
Answer: Minerals are primarily classified based on two main categories: physical properties and
chemical properties. The physical properties include luster, streak, crystal form, hardness, density, and
cleavage or breakage. Chemical properties involve the mineral’s chemical composition and reactions with
other substances. These characteristics help identify and distinguish different minerals in mineralogy.

 """},
{'question':
 """
 Question: How do you find theasymptotes for Q(x) = (2x
2
)/(x
2 − 5x − 6)?
Answer: vertical asymptotes x=-1, x=6/n horizontal asymptote y=2 /n /n /n /n /n /n /n /n /n /n

 """,
'answer':
 """
 Question: How do you determine the vertical and horizontal asymptotes for the function Q(x) =
2x
2
x2−5x−6
?
Answer: To find the vertical asymptotes, we need to identify the values of x that make the denominator
equal to zero:
x
2 − 5x − 6 = (x − 6)(x + 1) = 0
This gives us the vertical asymptotes at x = −1 and x = 6.
For the horizontal asymptote, as x approaches infinity or negative infinity, the dominant terms in the
numerator and denominator are both x
2
. Thus, we compare their coefficients:
- Numerator’s highest degree coefficient: 2 - Denominator’s highest degree coefficient: 1
The horizontal asymptote is given by the ratio of these coefficients:
y =
2
1
= 2
Therefore, the vertical asymptotes are at x = −1 and x = 6, and the horizontal asymptote is y = 2.
"""}
,{
    'question':"""
    Question:Which of the following is not an example of biased language? a. “You are too short to reach
that.” b. “Are you deaf?” c. “Move along, grandpa!” d. “I think I’ve been gypped.
Answer: D. I think I’ve been gypped.
    """,
    'answer':"""
Question: Which of the following statements does not exhibit biased language?
a. “You are too short to reach that.”
b. “Are you deaf?”
c. “Move along, grandpa!”
d. “I think I’ve been gypped.”
Answer: The statement that does not exhibit biased language is:
d. "I think I’ve been gypped."
Explanation: Biased language refers to words or phrases that stereotype, discriminate, or show prejudice
towards a particular group. In this case, options a, b, and c contain biases towards people’s physical
abilities (height), hearing ability, and age, respectively. However, "I think I’ve been gypped" is an
informal expression meaning to be cheated or deceived, which, while potentially offensive, does not
directly target a specific group based on attributes like race, gender, or disabilities.
    """
},
    {
    'question':"""
Question: I am stucked on the following challenge: "If the line determined by two distinct points
(x1, y1) and (x2, y2) is not vertical, and therefore has slope (y2 − y1)/(x2 − x1), show that the pointslope form of its equation is the same regardless of which point is used as the given point." Okay, we
can separate (x0, y0) from the form to get:
y(x2 − x1) − x(y2 − y1) = y0(x2 − x1) − x0(y2 − y1)
But how exclude this point (x0, y0) and leave only x, y, x1, y1, x2, y2 in the equation? UPDATE: There
is a solution for this challenge:
(y1 − y2)x + (x2 − x1)y = x2y1 − x1y2
From the answer I found that
y2(x − x1) − y1(x − x2) = y(x2 − x1)
... but why this is true?
Answer: Thanks to saulspatz, the solution is to simply show that, whether we are using (x1, y1) or
(x2, y2) as the given point, the equation does not change.
So both equations:
y − y1 = m(x − x1)
y − y2 = m(x − x2)
reduce to:
(y1 − y2)x + (x2 − x1)y = x2y1 − x1y2.
    """,
    'answer':"""
Question: Demonstrate that the point-slope form of the equation of a line determined by two distinct
points (x1, y1) and (x2, y2), where the line is not vertical, remains consistent regardless of which point
is chosen as the reference. Why does the form of the equation not change when different reference
points are used?
Answer: To address this challenge, we need to confirm that the point-slope equation of a line remains
consistent regardless of the point chosen to derive it. Given two points (x1, y1) and (x2, y2) with a
non-vertical line passing through them, the slope m of the line is y2−y1
x2−x1
.
The point-slope form of a line’s equation based on point (x1, y1) is:
y − y1 = m(x − x1)
Similarly, using point (x2, y2) gives:
y − y2 = m(x − x2)
These can be rewritten and compared to show their equivalence. Expanding both equations: - From the
first equation: y − y1 =
y2−y1
x2−x1
(x − x1) - From the second equation: y − y2 =
y2−y1
x2−x1
(x − x2)
We simplify these to find a common form. Multiplying both sides by (x2 − x1) and rearranging terms,
we find:
y2(x − x1) − y1(x − x2) = y(x2 − x1)
This confirms that both forms collapse to the same linear equation, showing the consistency of the
point-slope form irrespective of the chosen reference point. Thus, it is demonstrated that the structural
form of the equation remains unchanged, ensuring that the mathematical properties of the line—defined
by its slope and points through which it passes—are invariant to the choice of the reference point used
in its expression. This uniformity stems from the linearity and the constant slope defined by any two
points on a straight line.
    
    """
}
]#more example better

prompt_refine = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Now, do this for the Q_A_pairs below !Keep the language same with the Q-A pairs. Don't split the answer into step, keep it as a whole.Question: {Q_A_pairs}",
    prefix=prefix,
    input_variables=["Q_A_pairs"],
)


In [ ]:
refine_pipe = llm | StrOutputParser()

In [3]:
refine_result =[]
for doc in q_a_pairs:
    refind_message = refine_pipe.invoke(input = prompt_refine.format(Q_A_pairs=doc))
    refine_result.append(refind_message)

NameError: name 'q_a_pairs' is not defined

In [ ]:
with open('Q_A pairs.txt','w')  as f:
    for item in refine_result:
        f.write(item+'\n')